### Exploring the Global Coral Bleaching Database to extract any usefull tables.

In [102]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# Connect to SQLite database
conn = sqlite3.connect("Global_Coral_Bleaching_Database_SQLite_11_24_21.db")

# Create a cursor object to interact with the database
cursor = conn.cursor()

In [3]:
# Get the list of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print table names
print("Tables in the database:", [table[0] for table in tables])

Tables in the database: ['Authors_LUT', 'Bleaching_Level_LUT', 'Bleaching_Prevalence_Score_LUT', 'Bleaching_tbl', 'City_Town_Name_LUT', 'Country_Name_LUT', 'Cover_tbl', 'Data_Source_LUT', 'Ecoregion_Name_LUT', 'Environmental_tbl', 'Exposure_LUT', 'Ocean_Name_LUT', 'R_Scripts_tbl', 'Realm_Name_LUT', 'Sample_Event_tbl', 'Severity_Code_LUT', 'Site_Info_tbl', 'State_Island_Province_Name_LUT', 'Substrate_Type_LUT', 'Query_2_Sample_Event_Counts', 'Query_3_Time_Series', 'Query_4_Samples_by_Data_Source', 'Query_5_Sites_by_Data_Source', 'Query_6_Sites_with_Multiple_Sample_Events', 'Subquery_1_Sites_and_Sample_Events', 'Subquery_6_Calculated_Reef_Check_Segments']


In [7]:
def convert_table_to_df(table_name, df_name):
    # Load the data into a pandas DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

    # Export to CSV
    df.to_csv(f"{df_name}.csv", index=False)
    print("Data exported to output")

In [6]:
convert_table_to_df('Site_Info_tbl', 'site_info')

Data exported to output.csv


In [9]:
convert_table_to_df('Cover_tbl', 'coral_cover')
convert_table_to_df('Bleaching_tbl', 'bleaching_info')
convert_table_to_df('Environmental_tbl', 'environment_info')

Data exported to output
Data exported to output
Data exported to output


In [54]:
convert_table_to_df('Sample_Event_tbl', 'sapme_info')

Data exported to output


## Taking a good look at the tables
### 1. Site Info

In [25]:
site_df = pd.read_csv('site_info.csv')
site_df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,City_Town_Name_2,City_Town_Name_3,City_Town_Name_4,Site_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Comments,TRIAL534
0,1,5,-14.280000,-170.715000,1,2,90.0,94.0,20.0,3977.0,960.0,1038.0,1039.0,b'',105.68,0.0,0.02450,52.544000,Staghorn corals (Acropora formosa = A. muricat...,T
1,2,5,-14.330000,-170.500000,1,2,90.0,94.0,20.0,3977.0,960.0,304.0,NaN,Slope,6923.20,2.0,0.02510,54.151228,b'',T
2,3,4,18.248389,-63.051139,2,7,45.0,2.0,78.0,NaN,NaN,NaN,NaN,b'',135.51,2.0,0.00000,86.047967,b'',T
3,4,4,-19.148333,146.870278,1,1,17.0,4.0,394.0,2218.0,180.0,NaN,NaN,b'',123.51,1.0,0.18185,43.390537,b'',T
4,5,4,-19.123778,146.878500,1,1,17.0,4.0,394.0,2218.0,1083.0,NaN,NaN,b'',20.92,1.0,0.00000,43.390537,b'',T


In this table Ocean_Name, Ecoregion_Name, Country_Name, State_Island_Province_Name, City_Town_Name need to be deciphered from other tables. So, let's do that.

In [26]:
oceans = pd.read_sql_query("SELECT * FROM Ocean_Name_LUT", conn)
oceans

,Ocean_ID,Ocean_Name,TRIAL524
0,1,Pacific,T
1,2,Atlantic,T
2,3,Indian,T
3,4,Red Sea,T
4,5,Arabian Gulf,T


In [27]:
site_df['Ocean_Name'].value_counts()

Ocean_Name
2    7222
1    5625
3    1199
4     231
5     128
Name: count, dtype: int64

In [28]:
site_df['Ocean_Name'] = site_df['Ocean_Name'].replace({1:'Pacific', 2:'Atlantic', 3:'Indian', 4:'Red Sea', 5:'Arabian Gulf'})
site_df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,City_Town_Name_2,City_Town_Name_3,City_Town_Name_4,Site_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Comments,TRIAL534
0,1,5,-14.280000,-170.715000,Pacific,2,90.0,94.0,20.0,3977.0,960.0,1038.0,1039.0,b'',105.68,0.0,0.02450,52.544000,Staghorn corals (Acropora formosa = A. muricat...,T
1,2,5,-14.330000,-170.500000,Pacific,2,90.0,94.0,20.0,3977.0,960.0,304.0,NaN,Slope,6923.20,2.0,0.02510,54.151228,b'',T
2,3,4,18.248389,-63.051139,Atlantic,7,45.0,2.0,78.0,NaN,NaN,NaN,NaN,b'',135.51,2.0,0.00000,86.047967,b'',T
3,4,4,-19.148333,146.870278,Pacific,1,17.0,4.0,394.0,2218.0,180.0,NaN,NaN,b'',123.51,1.0,0.18185,43.390537,b'',T
4,5,4,-19.123778,146.878500,Pacific,1,17.0,4.0,394.0,2218.0,1083.0,NaN,NaN,b'',20.92,1.0,0.00000,43.390537,b'',T


In [29]:
realms = pd.read_sql_query("SELECT * FROM Realm_Name_LUT", conn)
realms = realms.drop(['TRIAL524'], axis=1)
realms

,Realm_ID,Realm_Name
0,1,Central Indo-Pacific
1,2,Eastern Indo-Pacific
2,3,Temperate Australasia
3,4,Temperate Northern Atlantic
4,5,Temperate Northern Pacific
5,6,Temperate Southern Africa
6,7,Tropical Atlantic
7,8,Tropical Eastern Pacific
8,9,Western Indo-Pacific


In [30]:
realm_map = dict(zip(realms['Realm_ID'], realms['Realm_Name']))
site_df['Realm_Name'] = site_df['Realm_Name'].map(realm_map)
site_df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,City_Town_Name_2,City_Town_Name_3,City_Town_Name_4,Site_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Comments,TRIAL534
0,1,5,-14.280000,-170.715000,Pacific,Eastern Indo-Pacific,90.0,94.0,20.0,3977.0,960.0,1038.0,1039.0,b'',105.68,0.0,0.02450,52.544000,Staghorn corals (Acropora formosa = A. muricat...,T
1,2,5,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,90.0,94.0,20.0,3977.0,960.0,304.0,NaN,Slope,6923.20,2.0,0.02510,54.151228,b'',T
2,3,4,18.248389,-63.051139,Atlantic,Tropical Atlantic,45.0,2.0,78.0,NaN,NaN,NaN,NaN,b'',135.51,2.0,0.00000,86.047967,b'',T
3,4,4,-19.148333,146.870278,Pacific,Central Indo-Pacific,17.0,4.0,394.0,2218.0,180.0,NaN,NaN,b'',123.51,1.0,0.18185,43.390537,b'',T
4,5,4,-19.123778,146.878500,Pacific,Central Indo-Pacific,17.0,4.0,394.0,2218.0,1083.0,NaN,NaN,b'',20.92,1.0,0.00000,43.390537,b'',T


In [31]:
ecoregions = pd.read_sql_query("SELECT * FROM Ecoregion_Name_LUT", conn)
ecoregions = ecoregions.drop(['TRIAL501'], axis=1)
ecoregions

,Ecoregion_ID,Ecoregion_Name
0,1,Andaman Islands
1,2,Andaman Sea
2,3,"Arnhem Land, north Australia"
3,4,"Ashmore Reef, north-west Australia"
4,5,"Austral Islands, French Polynesia"
...,...,...
112,113,Vanuatu
113,114,West Sumatra
114,115,Western Mexico and Revillagigedo Islands
115,116,"Western Tuamotu Archipelago, central Pacific"


In [42]:
site_df['Ecoregion_Name'].unique()

array([ 90.,  45.,  17.,  67.,   4.,  70., 106.,  28.,  66.,  25.,  88.,
         3.,  22.,  48., 112.,  91.,  20.,  89.,   6.,  85.,  75.,   9.,
        10., 110.,  13.,  27.,  41.,  19.,  29.,  96.,  43.,  46.,  49.,
        23.,  60.,  63.,  24.,  21.,  26.,  36.,  56.,  34.,  71., 100.,
        87., 117.,  14.,  32.,  62.,  93., 116.,   5.,  78.,  39.,   1.,
        59.,  55.,  69.,  51.,  58.,  97.,   7., 114.,  16.,  42., 108.,
        15., 109.,  11.,  40.,  50., 101.,  77., 104.,  92.,  72.,  47.,
        nan,  83.,  53.,  35.,  86.,  73.,  98.,   2.,  61.,  37., 115.,
         8.,  99.,  74.,  68.,  81.,  84.,  44.,  65.,  12., 105.,  76.,
        38., 107.,  80.,  95., 113.,  30., 102., 111.,  33.,  31.,  57.,
        52.,  82., 103.,  79.,  18.,  94.,  54.])

In [38]:
eco_map = dict(zip(ecoregions['Ecoregion_ID'], ecoregions['Ecoregion_Name']))
eco_map

{1: 'Andaman Islands',
 2: 'Andaman Sea',
 3: 'Arnhem Land, north Australia',
 4: 'Ashmore Reef, north-west Australia',
 5: 'Austral Islands, French Polynesia',
 6: 'Bahamas and Florida Keys',
 7: 'Banda Sea and Molucca Islands',
 8: 'Bay of Campeche, Yucatan, Gulf of Mexico',
 9: 'Belize and west Caribbean',
 10: 'Bermuda',
 11: 'Birds Head Peninsula, Papua',
 12: 'Bismarck Sea, New Guinea',
 13: 'Brazil',
 14: 'Caroline Islands, Micronesia',
 15: 'Celebes Sea',
 16: 'Cenderawasih Bay, Papua',
 17: 'Central and northern Great Barrier Reef',
 18: 'Central Vietnam',
 19: 'Chagos Archipelago',
 20: 'Christmas Island, Indian Ocean',
 21: 'Cocos Island, Costa Rica',
 22: 'Cocos Keeling Atolls, Indian Ocean',
 23: 'Colombia, Ecuador and Chile, Pacific coast',
 24: 'Cook Islands, south-west Pacific',
 25: 'Coral Sea',
 26: 'Costa Rica and Panama, Pacific coast',
 27: 'Cuba and Cayman Islands',
 28: 'Darwin, north Australia',
 29: 'Easter Island, south central Pacific',
 30: 'Eastern coast So

In [43]:
# Convert NA to 0
eco_map = dict(zip(ecoregions['Ecoregion_ID'], ecoregions['Ecoregion_Name']))
site_df['Ecoregion_Name'] = site_df['Ecoregion_Name'].fillna(0).astype(int).map(eco_map)
site_df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,City_Town_Name_2,City_Town_Name_3,City_Town_Name_4,Site_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Comments,TRIAL534
0,1,5,-14.280000,-170.715000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",94.0,20.0,3977.0,960.0,1038.0,1039.0,b'',105.68,0.0,0.02450,52.544000,Staghorn corals (Acropora formosa = A. muricat...,T
1,2,5,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",94.0,20.0,3977.0,960.0,304.0,NaN,Slope,6923.20,2.0,0.02510,54.151228,b'',T
2,3,4,18.248389,-63.051139,Atlantic,Tropical Atlantic,"Hispaniola, Puerto Rico and Lesser Antilles",2.0,78.0,NaN,NaN,NaN,NaN,b'',135.51,2.0,0.00000,86.047967,b'',T
3,4,4,-19.148333,146.870278,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,4.0,394.0,2218.0,180.0,NaN,NaN,b'',123.51,1.0,0.18185,43.390537,b'',T
4,5,4,-19.123778,146.878500,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,4.0,394.0,2218.0,1083.0,NaN,NaN,b'',20.92,1.0,0.00000,43.390537,b'',T


In [46]:
countries = pd.read_sql_query("SELECT * FROM Country_Name_LUT", conn)
countries = countries.drop(['TRIAL495'], axis=1)
countries

,Country_ID,Country_Name
0,1,American Samoa
1,2,Anguilla
2,3,Antigua and Barbuda
3,4,Australia
4,5,Bahamas
...,...,...
91,96,Venezuela
92,97,Vietnam
93,98,Yemen
94,99,France


In [47]:
site_df['Country_Name'].unique()

array([ 94.,   2.,   4.,   3.,  62.,  nan,   5.,   6.,   7.,   8.,   9.,
        10.,  52.,  12., 100.,  16.,  14.,  15.,  18.,  70.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  99.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  53.,  54.,
        55.,  56.,  58.,  59.,  60.,  61.,  63.,  65.,  66.,  67.,  68.,
        69.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  81.,
        82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,
        93.,  95.,  96.,  97.,  98.])

In [48]:
country_map = dict(zip(countries['Country_ID'], countries['Country_Name']))
site_df['Country_Name'] = site_df['Country_Name'].fillna(0).astype(int).map(country_map)
site_df.head()

,Site_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,City_Town_Name_2,City_Town_Name_3,City_Town_Name_4,Site_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Comments,TRIAL534
0,1,5,-14.280000,-170.715000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",United States,20.0,3977.0,960.0,1038.0,1039.0,b'',105.68,0.0,0.02450,52.544000,Staghorn corals (Acropora formosa = A. muricat...,T
1,2,5,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",United States,20.0,3977.0,960.0,304.0,NaN,Slope,6923.20,2.0,0.02510,54.151228,b'',T
2,3,4,18.248389,-63.051139,Atlantic,Tropical Atlantic,"Hispaniola, Puerto Rico and Lesser Antilles",Anguilla,78.0,NaN,NaN,NaN,NaN,b'',135.51,2.0,0.00000,86.047967,b'',T
3,4,4,-19.148333,146.870278,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,Australia,394.0,2218.0,180.0,NaN,NaN,b'',123.51,1.0,0.18185,43.390537,b'',T
4,5,4,-19.123778,146.878500,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,Australia,394.0,2218.0,1083.0,NaN,NaN,b'',20.92,1.0,0.00000,43.390537,b'',T


In [49]:
site_df = site_df[['Site_ID', 'Latitude_Degrees', 'Longitude_Degrees', 'Ocean_Name', 'Realm_Name', 'Ecoregion_Name', 'Country_Name', 'Distance_to_Shore', 'Exposure', 'Turbidity', 'Cyclone_Frequency']]
site_df.head()

,Site_ID,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency
0,1,-14.280000,-170.715000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",United States,105.68,0.0,0.02450,52.544000
1,2,-14.330000,-170.500000,Pacific,Eastern Indo-Pacific,"Samoa, Tuvalu and Tonga",United States,6923.20,2.0,0.02510,54.151228
2,3,18.248389,-63.051139,Atlantic,Tropical Atlantic,"Hispaniola, Puerto Rico and Lesser Antilles",Anguilla,135.51,2.0,0.00000,86.047967
3,4,-19.148333,146.870278,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,Australia,123.51,1.0,0.18185,43.390537
4,5,-19.123778,146.878500,Pacific,Central Indo-Pacific,Central and northern Great Barrier Reef,Australia,20.92,1.0,0.00000,43.390537


In [50]:
site_df.shape

(14405, 11)

In [52]:
site_df.to_csv("site_info.csv", index=False)

### 2. Sample Info

In [58]:
sample = pd.read_csv('sapme_info.csv')
sample.head()

,Sample_ID,Site_ID,Reef_ID,Date_Day,Date_Month,Date_Year,Depth_m,Quadrat_No,Comments,TRIAL528
0,9623,4606,b'',8.0,9.0,2016.0,5.0,NaN,b'',T
1,9624,4607,b'',2.0,9.0,2016.0,10.0,NaN,b'',T
2,9625,4608,b'',29.0,6.0,2010.0,5.0,NaN,b'',T
3,9626,4609,b'',5.0,8.0,2008.0,5.0,NaN,b'',T
4,9627,4610,b'',2.0,9.0,2016.0,13.0,NaN,b'',T


In [61]:
sample['Comments'].value_counts()

Comments
b''                                                                                                                                                                                                                                                                4864
% bleached = Bleaching Index                                                                                                                                                                                                                                        157
Number of colonies observed: 7\n; Number of bleached colonies: 1                                                                                                                                                                                                     47
Number of colonies observed: 9\n; Number of bleached colonies: 2                                                                                                                                       

In [62]:
sample = sample[['Sample_ID', 'Site_ID', 'Reef_ID', 'Date_Day',	'Date_Month', 'Date_Year', 'Depth_m', 'Quadrat_No']]
sample.shape

(27008, 8)

In [64]:
samples = pd.merge(sample, site_df, on='Site_ID')
samples.head()

,Sample_ID,Site_ID,Reef_ID,Date_Day,Date_Month,Date_Year,Depth_m,Quadrat_No,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency
0,9623,4606,b'',8.0,9.0,2016.0,5.0,NaN,24.366926,124.003372,Pacific,Central Indo-Pacific,"South Ryukyu Islands, Japan",Japan,248.75,0.0,0.0624,65.578313
1,9624,4607,b'',2.0,9.0,2016.0,10.0,NaN,24.454807,124.109802,Pacific,Central Indo-Pacific,"South Ryukyu Islands, Japan",Japan,810.55,0.0,0.0422,50.228451
2,9625,4608,b'',29.0,6.0,2010.0,5.0,NaN,24.457072,124.098558,Pacific,Central Indo-Pacific,"South Ryukyu Islands, Japan",Japan,434.38,2.0,0.0422,50.228451
3,9626,4609,b'',5.0,8.0,2008.0,5.0,NaN,24.463401,123.802185,Pacific,Central Indo-Pacific,"South Ryukyu Islands, Japan",Japan,1626.36,1.0,0.0391,77.111338
4,9627,4610,b'',2.0,9.0,2016.0,13.0,NaN,24.482618,124.125423,Pacific,Central Indo-Pacific,"South Ryukyu Islands, Japan",Japan,372.30,1.0,0.0417,50.228451


In [65]:
sample.to_csv('sample_info.csv', index=False)

### 3. Coral Cover

In [76]:
coral = pd.read_csv('coral_cover.csv')
coral.head()

,Cover_ID,Sample_ID,Percent_Hard_Coral,Percent_Macroalgae,Substrate_Type,S1,S2,S3,S4,Comments,TRIAL495
0,22558,10290512,9.47,41.84,NaN,NaN,NaN,NaN,NaN,NaN,T
1,22559,10290513,14.76,51.43,NaN,NaN,NaN,NaN,NaN,NaN,T
2,22560,10290514,65.50,11.60,NaN,NaN,NaN,NaN,NaN,NaN,T
3,22561,10290515,39.38,21.88,NaN,NaN,NaN,NaN,NaN,NaN,T
4,22562,10290516,48.33,16.67,NaN,NaN,NaN,NaN,NaN,NaN,T


In [77]:
subs_type = pd.read_sql_query("SELECT * FROM Substrate_Type_LUT", conn)
subs_type = subs_type.drop(['TRIAL541'], axis=1)
subs_type

,Substrate_ID,Substrate_Name
0,1,Hard Coral
1,2,Fleshy Seaweed
2,3,Nutrient Indicator Algae


In [78]:
coral['Substrate_Type'].value_counts()

Substrate_Type
1.0    14359
3.0    11979
2.0     2379
Name: count, dtype: int64

In [79]:
coral.shape

(31310, 11)

In [80]:
subs_map = dict(zip(subs_type['Substrate_ID'], subs_type['Substrate_Name']))
coral['Substrate_Type'] = coral['Substrate_Type'].fillna(0).astype(int).map(subs_map)
coral.head()

,Cover_ID,Sample_ID,Percent_Hard_Coral,Percent_Macroalgae,Substrate_Type,S1,S2,S3,S4,Comments,TRIAL495
0,22558,10290512,9.47,41.84,NaN,NaN,NaN,NaN,NaN,NaN,T
1,22559,10290513,14.76,51.43,NaN,NaN,NaN,NaN,NaN,NaN,T
2,22560,10290514,65.50,11.60,NaN,NaN,NaN,NaN,NaN,NaN,T
3,22561,10290515,39.38,21.88,NaN,NaN,NaN,NaN,NaN,NaN,T
4,22562,10290516,48.33,16.67,NaN,NaN,NaN,NaN,NaN,NaN,T


In [81]:
coral['Substrate_Type'].value_counts()

Substrate_Type
Hard Coral                  14359
Nutrient Indicator Algae    11979
Fleshy Seaweed               2379
Name: count, dtype: int64

In [82]:
coral['S1'].value_counts()

S1
0.0     9026
1.0     1904
2.0     1361
3.0     1195
4.0     1033
6.0      911
5.0      887
7.0      842
8.0      834
9.0      791
10.0     756
11.0     717
12.0     711
13.0     651
14.0     634
15.0     620
16.0     575
18.0     525
17.0     499
19.0     471
20.0     403
21.0     376
22.0     367
23.0     329
25.0     305
24.0     300
26.0     253
27.0     242
28.0     195
29.0     175
30.0     144
31.0     132
32.0     115
33.0      91
35.0      70
34.0      69
36.0      51
37.0      35
38.0      20
39.0      15
40.0      11
Name: count, dtype: int64

In [83]:
coral['S2'].value_counts()

S2
0.0     8956
1.0     1850
2.0     1455
3.0     1280
4.0     1038
5.0      959
6.0      914
7.0      845
8.0      838
9.0      780
11.0     723
10.0     707
12.0     707
13.0     636
14.0     620
16.0     614
15.0     570
18.0     509
17.0     480
19.0     459
20.0     410
21.0     407
22.0     391
24.0     333
23.0     315
25.0     277
26.0     256
27.0     219
28.0     208
29.0     165
30.0     156
31.0     122
32.0      98
33.0      81
34.0      68
35.0      59
36.0      52
37.0      27
38.0      21
39.0      11
40.0      11
Name: count, dtype: int64

In [84]:
coral = coral[['Cover_ID', 'Sample_ID',	'Percent_Hard_Coral', 'Percent_Macroalgae',	'Substrate_Type', 'S1',	'S2', 'S3',	'S4']]
coral.shape

(31310, 9)

In [85]:
coral.to_csv('coral_cover.csv', index=False)

### 4. Bleeching Info

In [87]:
bleaching = pd.read_csv('bleaching_info.csv')
bleaching.head()

,Bleaching_ID,Sample_ID,Bleaching_Level,S1,S2,S3,S4,Percent_Bleaching_Old_Method,Severity_Code,Percent_Bleached,Number__Bleached_Colonies,bleach_intensity,Bleaching_Prevalence_Score,TRIAL485
0,9623,9623,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T
1,9624,9624,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T
2,9625,9625,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,T
3,9626,9626,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,T
4,9627,9627,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T


In [96]:
bleaching['Bleaching_Level'].value_counts()

Bleaching_Level
Population    11297
Colony        11028
Name: count, dtype: int64

In [94]:
bleach_level = pd.read_sql_query("SELECT * FROM Bleaching_Level_LUT", conn)
bleach_level

,Bleaching_Level_ID,Bleaching_Level,TRIAL485
0,1,Population,T
1,2,Colony,T


In [95]:
level_map = dict(zip(bleach_level['Bleaching_Level_ID'], bleach_level['Bleaching_Level']))
bleaching['Bleaching_Level'] = bleaching['Bleaching_Level'].fillna(0).astype(int).map(level_map)
bleaching.head()

,Bleaching_ID,Sample_ID,Bleaching_Level,S1,S2,S3,S4,Percent_Bleaching_Old_Method,Severity_Code,Percent_Bleached,Number__Bleached_Colonies,bleach_intensity,Bleaching_Prevalence_Score,TRIAL485
0,9623,9623,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T
1,9624,9624,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T
2,9625,9625,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,T
3,9626,9626,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,T
4,9627,9627,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,T


In [97]:
severity_level = pd.read_sql_query("SELECT * FROM Severity_Code_LUT", conn)
severity_level

,Severity_ID,Severity_Code,TRIAL531
0,-1,% unknown,T
1,0,No Bleaching,T
2,1,Mild (1-10% Bleached),T
3,2,Moderate (11-50% Bleached),T
4,3,Severe (>50% Bleached),T


In [98]:
level_map = dict(zip(severity_level['Severity_ID'], severity_level['Severity_Code']))
bleaching['Severity_Code'] = bleaching['Severity_Code'].fillna(0).astype(int).map(level_map)
bleaching.head()

,Bleaching_ID,Sample_ID,Bleaching_Level,S1,S2,S3,S4,Percent_Bleaching_Old_Method,Severity_Code,Percent_Bleached,Number__Bleached_Colonies,bleach_intensity,Bleaching_Prevalence_Score,TRIAL485
0,9623,9623,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN,T
1,9624,9624,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN,T
2,9625,9625,NaN,NaN,NaN,NaN,NaN,NaN,No Bleaching,NaN,NaN,NaN,NaN,T
3,9626,9626,NaN,NaN,NaN,NaN,NaN,NaN,Moderate (11-50% Bleached),NaN,NaN,NaN,NaN,T
4,9627,9627,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN,T


In [99]:
bleaching = bleaching.drop(['TRIAL485'], axis=1)
bleaching.head()

,Bleaching_ID,Sample_ID,Bleaching_Level,S1,S2,S3,S4,Percent_Bleaching_Old_Method,Severity_Code,Percent_Bleached,Number__Bleached_Colonies,bleach_intensity,Bleaching_Prevalence_Score
0,9623,9623,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN
1,9624,9624,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN
2,9625,9625,NaN,NaN,NaN,NaN,NaN,NaN,No Bleaching,NaN,NaN,NaN,NaN
3,9626,9626,NaN,NaN,NaN,NaN,NaN,NaN,Moderate (11-50% Bleached),NaN,NaN,NaN,NaN
4,9627,9627,NaN,NaN,NaN,NaN,NaN,NaN,Severe (>50% Bleached),NaN,NaN,NaN,NaN


In [100]:
bleaching.to_csv('bleaching_info.csv', index=False)

### 5. Environment Info

In [101]:
env_df = pd.read_csv('environment_info.csv')
env_df.head()

,Environmental_ID,Sample_ID,ClimSST,Temperature_Kelvin,Temperature_Mean,Temperature_Minimum,Temperature_Maximum,Temperature_Kelvin_Standard_Deviation,Windspeed,SSTA,...,TSA_Mean,TSA_Frequency,TSA_Frequency_Standard_Deviation,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,TRIAL501
0,9622,9623,299.48,301.30,298.85,293.85,304.38,2.48,8.0,-0.21,...,-3.36,6.0,1.26,6.0,1.0,6.88,0.95,6.88,0.35,T
1,9623,9624,299.92,302.17,298.93,294.13,305.10,2.46,5.0,0.45,...,-3.26,4.0,1.42,6.0,1.0,8.24,1.44,8.23,0.52,T
2,9624,9625,302.20,300.94,298.93,294.13,305.10,2.46,6.0,-0.50,...,-3.26,0.0,1.42,6.0,1.0,0.00,1.44,8.23,0.52,T
3,9625,9626,301.76,300.26,299.09,293.98,305.42,2.42,5.0,-2.06,...,-3.22,0.0,1.21,6.0,1.0,0.00,1.13,7.82,0.42,T
4,9626,9627,299.86,301.40,298.92,294.25,305.51,2.46,5.0,-0.27,...,-3.27,6.0,1.66,6.0,1.0,10.78,1.65,10.78,0.59,T


In [103]:
conn.close()